# ガウス過程

COMBOではガウス過程回帰を実行しながらベイズ最適化を行なっている。

そのため、学習データが与えられた際にガウス過程回帰を実行することもでき、学習済みモデルを利用したテストデータの予測も行うことができる。

ここでは、その手順について紹介する。


## 探索候補データの準備

これまでのマニュアルでも利用していたs5-210.csvを用いてガウス過程を実行します。

In [11]:
import combo

import os
import urllib
import ssl
import numpy as np

ssl._create_default_https_context = ssl._create_unverified_context

def download():
    if not os.path.exists('data/s5-210.csv'):

        if not os.path.exists('data'):
            os.mkdir('data')
            
        print('Downloading...')
        with urllib.request.urlopen("http://www.tsudalab.org/files/s5-210.csv") as response, open('data/s5-210.csv', 'wb') as out_file:
            out_file.write(response.read())
        print('Done')
        
def load_data():
    #download()
    A =  np.asarray(np.loadtxt('data/s5-210.csv',skiprows=1,delimiter=',') )
    X = A[:,0:3]
    t  = -A[:,3]
    return X, t

X, t = load_data()
X = combo.misc.centering( X )


## 学習データの定義

対象データのうち、８割をトレーニングデータとして利用し、２割をテストデータとして利用できるように分割します。

In [12]:
id_all   = np.random.choice(len(t), len(t), replace=False)
id_train  = id_all[0:int(len(t)*0.1)]
id_test = id_all[int(len(t)*0.8):len(t)]

X_train = X[id_train]
X_test = X[id_test]

t_train = t[id_train]
t_test = t[id_test]


## ガウス過程の学習と予測

以下のプロセスでガウス過程を学習し、テストデータの予測を行う。

１、ガウス過程のモデルを生成する。

２、X_train, t_trainを用いてモデルを学習する。

３、学習されたモデルを用いてテストデータ(X_test)に対する予測を実行する。

In [15]:
#共分散の定義(ガウシアン)
cov = combo.gp.cov.gauss( X_train.shape[1],ard = False )

#平均の定義
mean = combo.gp.mean.const()

#尤度関数の定義(ガウシアン)
lik = combo.gp.lik.gauss()

#ガウス過程モデルの生成
gp = combo.gp.model(lik=lik,mean=mean,cov=cov)
config = combo.misc.set_config()



#ガウス過程モデルを学習
gp.fit(X_train, t_train, config)

#学習されたガウス過程におけるパラメタを出力
gp.print_params()



#テストデータの平均値（予測値）および分散を計算
gp.prepare(X_train, t_train)
fmean = gp.get_post_fmean(X_train, X_test)
fcov = gp.get_post_fcov(X_train, X_test)


Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0 -th epoch marginal likelihood 20116.31340799952
50 -th epoch marginal likelihood 7449.118866812681
100 -th epoch marginal likelihood 4448.950758973205
150 -th epoch marginal likelihood 2602.476438510648
200 -th epoch marginal likelihood 986.0295683274035
250 -th epoch marginal likelihood -176.85205456828135
300 -th epoch marginal likelihood -803.1601264779774
350 -th epoch marginal likelihood -1242.9555258335376
400 -th epoch marginal likelihood -1454.0324191553887
450 -th epoch marginal likelihood -1575.5402432541782
500 -th epoch marginal likelihood -1635.0787954316102
Done



likelihood parameter =   [-2.7509835]
mean parameter in GP prior:  [-1.059573]
covariance parameter in GP prior:  [-1.23719766 -2.51086395]




In [16]:
fmean

array([-1.14390157, -0.98930274, -0.98061332, ..., -1.08664574,
       -1.07799135, -1.05788019])

分散の結果

In [17]:
fcov

array([0.00099564, 0.00129194, 0.00144664, ..., 0.00118036, 0.00095491,
       0.00165648])

予測値の平均二乗誤差の出力

In [18]:
np.mean((fmean-t_test)**2)

0.007159063044483804

## 訓練済みモデルによる予測

学習済みモデルのパラメタをgp_paramsとして読み出し、これを用いた予測を行う。

gp_paramsおよび学習データ(X_train, t_train)を記憶しておくことで、訓練済みモデルによる予測が可能となる。

学習されたパラメタを準備(学習の直後に行う必要あり)

In [24]:
#学習したパラメタを１次元配列として準備
gp_params =  np.append(np.append(gp.lik.params, gp.prior.mean.params), gp.prior.cov.params)

gp_params

array([-2.7509835 , -1.059573  , -1.23719766, -2.51086395])

学習に利用したモデルと同様のモデルをgpとして準備

In [25]:
#共分散の定義(ガウシアン)
cov = combo.gp.cov.gauss( X_train.shape[1],ard = False )

#平均の定義
mean = combo.gp.mean.const()

#尤度関数の定義(ガウシアン)
lik = combo.gp.lik.gauss()

#ガウス過程モデルの生成
gp = combo.gp.model(lik=lik,mean=mean,cov=cov)


学習済みのパラメタをモデルに入力し予測を実行

In [26]:
#学習済みのパラメタをガウス過程に入力
gp.set_params(gp_params)


#テストデータの平均値（予測値）および分散を計算
gp.prepare(X_train, t_train)
fmean = gp.get_post_fmean(X_train, X_test)
fcov = gp.get_post_fcov(X_train, X_test)

予測の結果

In [27]:
fmean

array([-1.14390157, -0.98930274, -0.98061332, ..., -1.08664574,
       -1.07799135, -1.05788019])

分散の結果

In [28]:
fcov

array([0.00099564, 0.00129194, 0.00144664, ..., 0.00118036, 0.00095491,
       0.00165648])

予測値の平均二乗誤差の出力

In [29]:
np.mean((fmean-t_test)**2)

0.007159063044483804